In [21]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer

In [22]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open("intents.json").read())

intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ['']},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
    'you are made 

In [23]:
words = [] # all word in patterns
classes = [] # all tags
documents = [] # (patterns, tag)
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

In [24]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [25]:
training = []
outputEmpty = [0] * len(classes)
for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    # print("bag :", bag)
    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    # print("outputempty :",outputEmpty)
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [26]:
trainX = training[:, :len(words)]
trainY = training[:, len(words):]

trainX

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [27]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]), ), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

In [28]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [29]:
hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
81/81 [==============================] - 2s 7ms/step - loss: 3.5988 - accuracy: 0.0668
Epoch 2/200
81/81 [==============================] - 0s 4ms/step - loss: 3.4332 - accuracy: 0.1238
Epoch 3/200
81/81 [==============================] - 0s 4ms/step - loss: 3.2363 - accuracy: 0.1708
Epoch 4/200
81/81 [==============================] - 0s 4ms/step - loss: 2.9396 - accuracy: 0.2426
Epoch 5/200
81/81 [==============================] - 0s 4ms/step - loss: 2.6908 - accuracy: 0.2970
Epoch 6/200
81/81 [==============================] - 0s 4ms/step - loss: 2.4782 - accuracy: 0.3465
Epoch 7/200
81/81 [==============================] - 0s 4ms/step - loss: 2.2407 - accuracy: 0.4183
Epoch 8/200
81/81 [==============================] - 0s 4ms/step - loss: 1.9611 - accuracy: 0.4604
Epoch 9/200
81/81 [==============================] - 1s 6ms/step - loss: 1.8585 - accuracy: 0.4802
Epoch 10/200
81/81 [==============================] - 0s 4ms/step - loss: 1.6787 - accuracy: 0.5074
Epoch 11/

In [30]:
model.save("grf_bot.h5", hist)
print(hist)

c:\Users\Rizal\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
